In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#df = pd.read_csv('/kaggle/input/tencentforuse/edit')
df=pd.read_csv('/kaggle/input/newclustering/newclasstering.csv')
#nlp=pd.read_csv('/kaggle/input/nlp-for-ml/NLP.CSV')

In [2]:
import numpy as np
k = np.loadtxt('/kaggle/input/nlpforuse/a.csv',dtype=np.float,delimiter=' ',unpack=False)
k=k[:2941]
print(len(k))

In [3]:
df = df[['year','topic','status','1','2','play cat']]

In [4]:
df[['status']]=df[['status']].fillna(round(df.mean()))
df=df.fillna(0)

In [5]:
#from scipy import stats
#t=stats.zscore(df) 
#t=abs(t) 
#filtered_entries = (t < 3).all(axis=1)
#new_df = df[filtered_entries]
new_df=df

In [6]:
#y_mean_std=(new_df['play number'].mean(),new_df['play number'].std())
#normalized_df=(new_df-new_df.mean())/new_df.std()
new_df['status']=(new_df['status']-new_df['status'].mean())/new_df['status'].std()
new_df['1']=(new_df['1']-new_df['1'].mean())/new_df['1'].std()
new_df['2']=(new_df['2']-new_df['2'].mean())/new_df['2'].std()
#new_df['year'].value_counts()
#from sklearn.preprocessing import LabelBinarizer
#y = OneHotEncoder().fit_transform(x).toarray()
#print(y)
#new_df
#print(y_mean_std)
new_df

need to deleting outliners

In [7]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
BATCH_SIZE=8

In [8]:
from sklearn.preprocessing import LabelBinarizer
k=torch.tensor(k)
#print(k.size())
year= LabelBinarizer().fit_transform(new_df['year'])
year=torch.tensor(year)
topic=LabelBinarizer().fit_transform(new_df['topic'])
topic=torch.tensor(topic)
#print(topic[0])
#print(year.size())
x= torch.tensor(new_df.iloc[:, 2:-1].values)
#print(x)
y=torch.tensor(new_df.iloc[:, -1].values)
y = y.view(-1, 1)
#print(y)
data = torch.cat((x, year), 1)
data = torch.cat((data, topic), 1)
data = torch.cat((data, k), 1)
data = torch.cat((data, y), 1)
#print(data.size())
train, test = train_test_split(data, test_size=0.1)
#print(train.size()[1])
x_train, y_train=torch.split(train, [train.size()[1]-1,1],dim=1)
x_test, y_test=torch.split(test, [train.size()[1]-1,1],dim=1)
#year_train = LabelBinarizer().fit_transform(train['year'])
#x_train = torch.tensor(train.iloc[:, 0:-2].values)
#y_train=torch.tensor(train.iloc[:, -1].values)
#year_train=torch.tensor(year_train)
#x_train = torch.cat((x_train, year_train), 1)

#year_test = LabelBinarizer().fit_transform(test['year'])
#x_test = torch.tensor(test.iloc[:, 0:-2].values)
#y_test=torch.tensor(test.iloc[:, -1].values)
#year_test=torch.tensor(year_test)
#x_test = torch.cat((x_test, year_test), 1)

print(x_train.size())
#x_test

In [9]:
dataset1 = TensorDataset(x_train, y_train)
trainset = DataLoader(dataset1, batch_size=BATCH_SIZE, shuffle=True)
dataset2 = TensorDataset(x_test, y_test)
testset = DataLoader(dataset2, batch_size=len(y_test), shuffle=False)

In [10]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4483, 1024)
        self.fc2 = nn.Linear(1024, 256)
        #self.fc3 = nn.Linear(256, 128)
        #self.fc4 = nn.Linear(128, 128)
        #self.fc5 = nn.Linear(128, 128)
        #self.fc6 = nn.Linear(128, 128)
        #self.fc7 = nn.Linear(128, 128)
        self.fc8 = nn.Linear(256, 128)
        self.fc9 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        #x = F.relu(self.fc4(x))
        #x = F.relu(self.fc5(x))
        #x = F.relu(self.fc6(x))  
        #x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = self.fc9(x)
        return x

net = Net()
print(net)


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr=1e-3)
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [12]:
for epoch in range(200): # 3 full passes over the data
    for idx, data in enumerate(trainset):  # `data` is a batch of data
        #if idx > 0:
            #continue
        X, y = data    # X is the batch of features, y is the batch of targets.
        net.zero_grad()  # sets gradients to 0 before loss calc. You will do this likely every step.
        #print('x:',X)
        X=X.clone().detach().float()
        output = net(X.view(-1,4483))  # pass in the reshaped batch (recall they are 28x28 atm) view(-1,3)
        #print(y)
        #y = y.clone().detach().float()
        #loss = F.nll_loss(output, y.view(-1, 1))
        y = y.type(torch.LongTensor)
        y = y.squeeze(1) 
        #print(output,y)
        loss = criterion(output, y)  # calc and grab the loss value torch.nn.MSELoss
        #print(loss)
        loss.backward()  # apply this loss backwards thru the network's parameters
        optimizer.step()  # attempt to optimize weights to account for loss/gradients
    #print(output) 
    #print(output,y)
    if epoch % 5 == 0:
        #print(y,output)
        print(epoch,loss)  # print loss. We hope loss (a measure of wrong-ness) declines! 
    if loss<=0.01:
        break
print('end')
        

In [13]:
s=0
n=0
for data in testset:
    X,y=data
    X=X.clone().detach().float()
    output = net(X.view(-1,4483))  # pass in the reshaped batch (recall they are 28x28 atm) view(-1,3)
        #print(output)
        #print(y)
    #y = y.clone().detach().float()
    y = y.type(torch.LongTensor)
    y = y.squeeze(1) 
    output=torch.max(output, 1).indices
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    x=confusion_matrix(y_test, output)
    accuracy = accuracy_score(y_test, output)
    print(x)
    print(accuracy)